In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import os

In [3]:
def get_video_files(directory):
    video_files = []
    subfolders = ['deceptive', 'truthful']
    for subfolder in subfolders:
        path = os.path.join(directory, subfolder)
        files = [os.path.join(subfolder, f) for f in os.listdir(path) if f.endswith('_metrics.csv')]
        video_files.extend(files)
    return video_files

In [4]:
def load_data_from_files(file_list, directory):
    data_frames = []
    for file in file_list:
        path = os.path.join(directory, file)
        df = pd.read_csv(path)
        pd.get_dummies(df['Emotion']) # Must convert to one-hot encoding here to avoid issues with missing values/columns
        data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

In [5]:
video_metric_directory = '../video_metrics/'
video_files = get_video_files(video_metric_directory)
train_files, test_files = train_test_split(video_files, test_size=0.2, random_state=42)

train_data = load_data_from_files(train_files, video_metric_directory)
test_data = load_data_from_files(test_files, video_metric_directory)

C:\Users\cpfit\AppData\Local\Temp\ipykernel_6192\4119103708.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(data_frames, ignore_index=True)
C:\Users\cpfit\AppData\Local\Temp\ipykernel_6192\4119103708.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(data_frames, ignore_index=True)


In [6]:
train_data.drop(columns=['Video_Path'], inplace=True)
test_data.drop(columns=['Video_Path'], inplace=True)

train_data = train_data.fillna(train_data.median())
test_data = test_data.fillna(test_data.median())

C:\Users\cpfit\AppData\Local\Temp\ipykernel_6192\2728043661.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data = train_data.fillna(train_data.median())
C:\Users\cpfit\AppData\Local\Temp\ipykernel_6192\2728043661.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data = test_data.fillna(test_data.median())


In [7]:
X_train = train_data.drop(columns=['Label'])
y_train = train_data['Label']
X_test = test_data.drop(columns=['Label'])
y_test = test_data['Label']

# Logistic Regression

In [8]:
# Initialize and train the XGBoost model
log_reg_model = LogisticRegression()

In [9]:
# Define the parameters range for random search
param_dist = {
    'C': np.logspace(-4, 4, 20),  # Regularization strength
    'solver': ['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga'],  # Algorithms to use in the optimization problem
    'penalty': ['l2', 'elasticnet'],  # Norm used in the penalization
    'max_iter': [100, 200, 300, 400, 500]  # Maximum number of iterations taken for the solvers to converge
}

# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=log_reg_model, param_distributions=param_dist, n_iter=50, cv=5, verbose=2, random_state=42, n_jobs=-1)

In [10]:
# Fit the random search model
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Users\cpfit\OneDrive\Documents\GitHub\cv-lie-detection\venv\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
135 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\cpfit\OneDrive\Documents\GitHub\cv-lie-detection\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\cpfit\OneDrive\Documents\GitHub\cv-lie-detection\venv\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  F

RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=50, n_jobs=-1,
                   param_distributions={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                                        'max_iter': [100, 200, 300, 400, 500],
                                        'penalty': ['l2', 'elasticnet'],
                                        'solver': ['liblinear', 'lbfgs',
                                                   'newton-cg', 'sag',
                                                   'saga']},
                   random_state=42, verbose=2)

In [11]:
# Evaluate the best model on the test set
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Best parameters:", random_search.best_params_)
print("Best accuracy:", random_search.best_score_)
print("Test set accuracy:", accuracy)
print("Classification Report:\n", class_report)

Best parameters: {'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 400, 'C': 0.012742749857031334}
Best accuracy: 0.6449182492405552
Test set accuracy: 0.6237674874812864
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.76      0.68     10063
           1       0.65      0.48      0.55      9308

    accuracy                           0.62     19371
   macro avg       0.63      0.62      0.61     19371
weighted avg       0.63      0.62      0.62     19371

